### Transforming Canopy Heights
* Separating this for now from the other transformations due to the different methods of data collection for this trait
* Need tests to ensure the correct transformations

### Questions
* What is access level?

In [2]:
import datetime
import pandas as pd
import numpy as np
import sqlite3

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_0 = pd.read_csv('../data/raw/mac_season_4.csv')
# df_0.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18,26,32,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# for col in df_0.columns.values:
#     print(f'Number of unique variables in {col}: {df_0[col].nunique()}')

In [5]:
# df_0.columns

In [6]:
cols_to_keep = ['result_type', 'id', 'site_id', 'sitename', 'lat', 'lon', 'cultivar_id', 'author', 
                'date', 'time', 'raw_date', 'trait', 'trait_description', 'mean', 'units', 'notes', 
                'access_level', 'cultivar', 'entity', 'method_name']

In [7]:
cols_to_drop = []

for col in df_0.columns.values:
    
    if col not in cols_to_keep:
        cols_to_drop.append(col)

cols_to_drop

['Unnamed: 0',
 'checked',
 'citation_id',
 'treatment_id',
 'city',
 'scientificname',
 'commonname',
 'genus',
 'species_id',
 'citation_year',
 'treatment',
 'month',
 'year',
 'dateloc',
 'n',
 'statname',
 'stat',
 'view_url',
 'edit_url']

In [8]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
# df_1.head()

#### Slice dataset where `trait` == `canopy_height`

In [9]:
df_2 = df_1.loc[df_1.trait == 'canopy_height']
# df_2.head()

In [10]:
df_2.shape

(58056, 20)

In [10]:
# df_2.entity.unique()

In [11]:
# df_2.notes.unique()

#### Non-null `entity` and `notes` values are not associated with `canopy_height` so can be dropped 

In [11]:
df_3 = df_2.drop(labels=['entity', 'notes'], axis=1)
# df_3.head()

In [12]:
df_3.head()

,result_type,id,site_id,sitename,lat,lon,cultivar_id,author,date,time,raw_date,trait,trait_description,mean,units,access_level,cultivar,method_name
51339,traits,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,"Newcomb, Maria",2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",91.0,cm,4,PI329518,Manual canopy height
51340,traits,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",99.0,cm,4,PI152828,Manual canopy height
51341,traits,6001940853,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",63.0,cm,4,PI152828,Manual canopy height
75000,traits,6004768603,6000005801,MAC Field Scanner Season 4 Range 35 Column 14,33.075770,-111.974835,6000000989,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",22.0,cm,2,PI619838,3D scanner to 98th quantile height
75001,traits,6004768605,6000005295,MAC Field Scanner Season 4 Range 36 Column 2,33.075806,-111.975032,6000000707,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",26.0,cm,2,PI155885,3D scanner to 98th quantile height


#### Separate canopy height measurements
Differences include:
* `method_name`
    * Manual canopy height 
    * 3D scanner to 98th quantile height 
* `author`
    * Newcomb, Maria (manual)
    * Zongyang, Li (scanner)
* `time`
    * unspecified or unknown (manual)
    * 12:00:00 Arizona (scanner)
* `sitename`
    * _sometimes_ E and W specified (manual)
    * no E or W in sitename (scanner)

In [13]:
df_3.method_name.unique()

array(['Manual canopy height', '3D scanner to 98th quantile height'],
      dtype=object)

In [14]:
manual_df = df_3.loc[df_3.method_name == 'Manual canopy height']
manual_df.head()

,result_type,id,site_id,sitename,lat,lon,cultivar_id,author,date,time,raw_date,trait,trait_description,mean,units,access_level,cultivar,method_name
51339,traits,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,"Newcomb, Maria",2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",91.0,cm,4,PI329518,Manual canopy height
51340,traits,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",99.0,cm,4,PI152828,Manual canopy height
51341,traits,6001940853,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 May 29 (America/Phoenix),[time unspecified or unknown],2017-05-30 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",63.0,cm,4,PI152828,Manual canopy height
152562,traits,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,"Newcomb, Maria",2017 Jun 19 (America/Phoenix),[time unspecified or unknown],2017-06-20 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",91.0,cm,4,PI329518,Manual canopy height
152563,traits,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,"Newcomb, Maria",2017 Jun 4 (America/Phoenix),[time unspecified or unknown],2017-06-05 00:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",99.0,cm,4,PI152828,Manual canopy height


In [15]:
scanner_df = df_3.loc[df_3.method_name == '3D scanner to 98th quantile height']
scanner_df.head()

,result_type,id,site_id,sitename,lat,lon,cultivar_id,author,date,time,raw_date,trait,trait_description,mean,units,access_level,cultivar,method_name
75000,traits,6004768603,6000005801,MAC Field Scanner Season 4 Range 35 Column 14,33.075770,-111.974835,6000000989,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",22.0,cm,2,PI619838,3D scanner to 98th quantile height
75001,traits,6004768605,6000005295,MAC Field Scanner Season 4 Range 36 Column 2,33.075806,-111.975032,6000000707,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",26.0,cm,2,PI155885,3D scanner to 98th quantile height
75002,traits,6004768606,6000005294,MAC Field Scanner Season 4 Range 36 Column 3,33.075806,-111.975015,6000000562,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",33.0,cm,2,PI329333,3D scanner to 98th quantile height
75003,traits,6004768608,6000005292,MAC Field Scanner Season 4 Range 36 Column 5,33.075806,-111.974983,6000000726,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",23.0,cm,2,PI176766,3D scanner to 98th quantile height
75004,traits,6004768609,6000005840,MAC Field Scanner Season 4 Range 36 Column 6,33.075806,-111.974966,6000001056,"Zongyang, Li",2017 May 20,12:00:00 (America/Phoenix),2017-05-20 14:00:00 -0500,canopy_height,"top of the general canopy of the plant, discou...",21.0,cm,2,PI563355,3D scanner to 98th quantile height


In [18]:
for col in manual_df.columns:
    print(f'Number of unique values for {col}: {manual_df[col].nunique()}')

Number of unique values for result_type: 1
Number of unique values for id: 4866
Number of unique values for site_id: 1866
Number of unique values for sitename: 1866
Number of unique values for lat: 1866
Number of unique values for lon: 1866
Number of unique values for cultivar_id: 349
Number of unique values for author: 1
Number of unique values for date: 12
Number of unique values for time: 1
Number of unique values for raw_date: 12
Number of unique values for trait: 1
Number of unique values for trait_description: 1
Number of unique values for mean: 306
Number of unique values for units: 1
Number of unique values for access_level: 2
Number of unique values for cultivar: 349
Number of unique values for method_name: 1


In [19]:
for col in manual_df.columns:
    if manual_df[col].nunique() < 5:
        print(f'Unique values for {col}: {manual_df[col].unique()}')

Unique values for result_type: ['traits']
Unique values for author: ['Newcomb, Maria']
Unique values for time: ['[time unspecified or unknown]']
Unique values for trait: ['canopy_height']
Unique values for trait_description: ['top of the general canopy of the plant, discounting any exceptional branches, leaves or photosynthetic portions of the inflorescence.']
Unique values for units: ['cm']
Unique values for access_level: [4 2]
Unique values for method_name: ['Manual canopy height']


In [20]:
more_cols_to_drop_manual = ['result_type', 'author', 'time', 'trait', 'trait_description']

In [21]:
for col in scanner_df.columns:
    print(f'Number of unique values for {col}: {scanner_df[col].nunique()}')

Number of unique values for result_type: 1
Number of unique values for id: 32283
Number of unique values for site_id: 713
Number of unique values for sitename: 713
Number of unique values for lat: 713
Number of unique values for lon: 713
Number of unique values for cultivar_id: 351
Number of unique values for author: 1
Number of unique values for date: 69
Number of unique values for time: 1
Number of unique values for raw_date: 69
Number of unique values for trait: 1
Number of unique values for trait_description: 1
Number of unique values for mean: 406
Number of unique values for units: 1
Number of unique values for access_level: 2
Number of unique values for cultivar: 351
Number of unique values for method_name: 1


In [22]:
for col in scanner_df.columns:
    if scanner_df[col].nunique() < 5:
        print(f'Unique values for {col}: {scanner_df[col].unique()}')

Unique values for result_type: ['traits']
Unique values for author: ['Zongyang, Li']
Unique values for time: ['12:00:00 (America/Phoenix)']
Unique values for trait: ['canopy_height']
Unique values for trait_description: ['top of the general canopy of the plant, discounting any exceptional branches, leaves or photosynthetic portions of the inflorescence.']
Unique values for units: ['cm']
Unique values for access_level: [2 4]
Unique values for method_name: ['3D scanner to 98th quantile height']


In [23]:
more_cols_to_drop_manual

['result_type', 'author', 'time', 'trait', 'trait_description']

In [24]:
more_cols_to_drop = more_cols_to_drop_manual

In [25]:
scanner_df_1 = scanner_df.drop(labels=more_cols_to_drop, axis=1)
manual_df_1 = manual_df.drop(labels=more_cols_to_drop, axis=1)

print(scanner_df_1.shape)
print(manual_df_1.shape)

(53184, 13)
(4872, 13)


In [27]:
scanner_df_1.head()

,id,site_id,sitename,lat,lon,cultivar_id,date,raw_date,mean,units,access_level,cultivar,method_name
75000,6004768603,6000005801,MAC Field Scanner Season 4 Range 35 Column 14,33.075770,-111.974835,6000000989,2017 May 20,2017-05-20 14:00:00 -0500,22.0,cm,2,PI619838,3D scanner to 98th quantile height
75001,6004768605,6000005295,MAC Field Scanner Season 4 Range 36 Column 2,33.075806,-111.975032,6000000707,2017 May 20,2017-05-20 14:00:00 -0500,26.0,cm,2,PI155885,3D scanner to 98th quantile height
75002,6004768606,6000005294,MAC Field Scanner Season 4 Range 36 Column 3,33.075806,-111.975015,6000000562,2017 May 20,2017-05-20 14:00:00 -0500,33.0,cm,2,PI329333,3D scanner to 98th quantile height
75003,6004768608,6000005292,MAC Field Scanner Season 4 Range 36 Column 5,33.075806,-111.974983,6000000726,2017 May 20,2017-05-20 14:00:00 -0500,23.0,cm,2,PI176766,3D scanner to 98th quantile height
75004,6004768609,6000005840,MAC Field Scanner Season 4 Range 36 Column 6,33.075806,-111.974966,6000001056,2017 May 20,2017-05-20 14:00:00 -0500,21.0,cm,2,PI563355,3D scanner to 98th quantile height


#### Rename `mean` columns to `canopy_height`

In [29]:
scanner_df_2 = scanner_df_1.rename({'mean': 'canopy_height'}, axis=1)
manual_df_2 = manual_df_1.rename({'mean': 'canopy_height'}, axis=1)

# manual_df_2.head()

,id,site_id,sitename,lat,lon,cultivar_id,date,raw_date,canopy_height,units,access_level,cultivar,method_name
51339,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,2017 Jun 19 (America/Phoenix),2017-06-20 00:00:00 -0500,91.0,cm,4,PI329518,Manual canopy height
51340,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,2017 Jun 4 (America/Phoenix),2017-06-05 00:00:00 -0500,99.0,cm,4,PI152828,Manual canopy height
51341,6001940853,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,2017 May 29 (America/Phoenix),2017-05-30 00:00:00 -0500,63.0,cm,4,PI152828,Manual canopy height
152562,6001943979,6000007888,MAC Field Scanner Season 4 Range 30 Column 9 E,33.075590,-111.974913,6000000788,2017 Jun 19 (America/Phoenix),2017-06-20 00:00:00 -0500,91.0,cm,4,PI329518,Manual canopy height
152563,6001942254,6000007579,MAC Field Scanner Season 4 Range 20 Column 15 W,33.075231,-111.974823,6000000687,2017 Jun 4 (America/Phoenix),2017-06-05 00:00:00 -0500,99.0,cm,4,PI152828,Manual canopy height


In [30]:
# scanner_df_2.tail()

,id,site_id,sitename,lat,lon,cultivar_id,date,raw_date,canopy_height,units,access_level,cultivar,method_name
354995,6004797282,6000005865,MAC Field Scanner Season 4 Range 41 Column 10,33.075985,-111.974901,6000000969,2017 Aug 22,2017-08-22 14:00:00 -0500,392.0,cm,2,PI573193,3D scanner to 98th quantile height
354996,6004797283,6000005866,MAC Field Scanner Season 4 Range 41 Column 11,33.075985,-111.974884,6000000688,2017 Aug 22,2017-08-22 14:00:00 -0500,386.0,cm,2,PI152860,3D scanner to 98th quantile height
354997,6004797287,6000005882,MAC Field Scanner Season 4 Range 41 Column 15,33.075985,-111.974819,6000000953,2017 Aug 22,2017-08-22 14:00:00 -0500,342.0,cm,2,PI570087,3D scanner to 98th quantile height
354998,6004797288,6000005906,MAC Field Scanner Season 4 Range 42 Column 2,33.076021,-111.975032,6000000751,2017 Aug 22,2017-08-22 14:00:00 -0500,331.0,cm,2,PI257600,3D scanner to 98th quantile height
354999,6004797290,6000005888,MAC Field Scanner Season 4 Range 42 Column 4,33.076021,-111.974999,6000000225,2017 Aug 22,2017-08-22 14:00:00 -0500,370.0,cm,2,PI154844,3D scanner to 98th quantile height


#### Extract column and range values for additional columns

In [37]:
manual_df_3 = manual_df_2.copy()
scanner_df_3 = scanner_df_2.copy()

In [38]:
# this needs to be a function. . . 

manual_df_3['range'] = manual_df_3['sitename'].str.extract("Range (\d+)").astype(int)
manual_df_3['column'] = manual_df_3['sitename'].str.extract("Column (\d+)").astype(int)

scanner_df_3['range'] = scanner_df_3['sitename'].str.extract("Range (\d+)").astype(int)
scanner_df_3['column'] = scanner_df_3['sitename'].str.extract("Column (\d+)").astype(int)

In [41]:
# manual_df_3.sample(n=10)
# scanner_df_3.sample(n=10)

,id,site_id,sitename,lat,lon,cultivar_id,date,raw_date,canopy_height,units,access_level,cultivar,method_name,range,column
191718,6004790355,6000005643,MAC Field Scanner Season 4 Range 30 Column 12,33.075590,-111.974868,6000000910,2017 Jul 13,2017-07-13 14:00:00 -0500,267.0,cm,4,PI568717,3D scanner to 98th quantile height,30,12
230698,6004772496,6000005910,MAC Field Scanner Season 4 Range 43 Column 12,33.076057,-111.974868,6000000229,2017 Jun 6,2017-06-06 14:00:00 -0500,95.0,cm,2,PI506069,3D scanner to 98th quantile height,43,12
138511,6004778560,6000005737,MAC Field Scanner Season 4 Range 27 Column 8,33.075482,-111.974933,6000000899,2017 Jun 21,2017-06-21 14:00:00 -0500,219.0,cm,2,PI563295,3D scanner to 98th quantile height,27,8
159134,6004760856,6000006040,MAC Field Scanner Season 4 Range 47 Column 7,33.076201,-111.974950,6000000897,2017 May 2,2017-05-02 14:00:00 -0500,9.0,cm,2,PI563196,3D scanner to 98th quantile height,47,7
353707,6004794527,6000005386,MAC Field Scanner Season 4 Range 4 Column 14,33.074655,-111.974835,6000000221,2017 Aug 16,2017-08-16 14:00:00 -0500,354.0,cm,2,PI562730,3D scanner to 98th quantile height,4,14
158682,6004759988,6000005566,MAC Field Scanner Season 4 Range 13 Column 7,33.074979,-111.974950,6000000811,2017 May 1,2017-05-01 14:00:00 -0500,10.0,cm,2,PI329711,3D scanner to 98th quantile height,13,7
174304,6004787798,6000005326,MAC Field Scanner Season 4 Range 47 Column 15,33.076201,-111.974819,6000000227,2017 Jul 6,2017-07-06 14:00:00 -0500,217.0,cm,2,PI506114,3D scanner to 98th quantile height,47,15
79454,6004777456,6000006044,MAC Field Scanner Season 4 Range 47 Column 9,33.076201,-111.974917,6000000982,2017 Jun 19,2017-06-19 14:00:00 -0500,204.0,cm,2,PI585961,3D scanner to 98th quantile height,47,9
322356,6004765279,6000005306,MAC Field Scanner Season 4 Range 38 Column 7,33.075877,-111.974950,6000000722,2017 May 11,2017-05-11 14:00:00 -0500,15.0,cm,2,PI157804,3D scanner to 98th quantile height,38,7
172662,6004784215,6000005511,MAC Field Scanner Season 4 Range 31 Column 12,33.075626,-111.974868,6000000749,2017 Jun 30,2017-06-30 14:00:00 -0500,207.0,cm,2,PI255744,3D scanner to 98th quantile height,31,12


### For Future Tests

In [16]:
# if slicing df based on certain values, ensure that no rows were overlooked

print(manual_df.shape)
print(scanner_df.shape)
print(df_3.shape)

manual_df.shape[0] + scanner_df.shape[0] == df_3.shape[0]

(4872, 18)
(53184, 18)
(58056, 18)


True

In [ ]:
# test for column and range extraction

#### Convert to .csv

In [42]:
# to be intentional about my csv creations

do_you_need_to_create_a_csv_right_now = False

if do_you_need_to_create_a_csv_right_now:
    
    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'canopy_height_manual_{timestamp}.csv'.replace(':', '')
    manual_df_3.to_csv(f'../data/processed/{output_filename}')

In [43]:
do_you_need_to_create_another_csv_right_now = False

if do_you_need_to_create_another_csv_right_now:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'canopy_height_scanner_{timestamp}.csv'.replace(':', '')
    scanner_df_3.to_csv(f'../data/processed/{output_filename}')